In [0]:
import pandas as pd
from google.colab import drive

In [2]:
!pip install uszipcode

     |████████████████████████████████| 378kB 9.6MB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
     |████████████████████████████████| 122kB 42.0MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for autopep8: filename=autopep8-1.4.4-py2.py3-none-any.whl size=42631 sha256=7674ef3ac29c9f8b1fe27bc01853f207db715a4461a1b6091b5058b272300808
  Stored in directory: /root/.cache/pip/wheels/7e/f5/4b/c19e6276126325eb8071b273347c05a830c37a82b9b3b81510
Successfully built autopep8


In [3]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [4]:
park_df = pd.read_csv('https://data.cityofnewyork.us/resource/99ii-hwh9.csv')
# park_df.head(5)
# park_df.dtypes
# park_df.index

# Changing the format of the data in the table for easier extraction
i = 0
for zipcode in park_df['the_geom']:
  park_df.loc[park_df.index == i, 'the_geom'] = zipcode[7:len(zipcode) - 1]
  i += 1

park_df.head(5)

,the_geom,gispropnum,parkname,type
0,-73.8345470371915 40.64951548820048,NaN,GATEWAY,1/2-Mile
1,-73.83150930469213 40.64992288722615,NaN,GATEWAY,1/2-Mile
2,-73.83362089255507 40.649643900721536,NaN,GATEWAY,1/2-Mile
3,-73.83272434273998 40.64978363054011,NaN,GATEWAY,1/2-Mile
4,-73.83548803206288 40.649370165110476,NaN,GATEWAY,1/2-Mile


In [0]:
# extract zip codes
Zipcode = list()  # will be used as a new column
for code in park_df['the_geom']:
  longlat = code.split(' ')
  lattitude = float(longlat[1])
  longitude = float(longlat[0])
  result = search.by_coordinates(lattitude, longitude, radius=30, returns=5)
  Zipcode.append(str(result)[24:29])

In [6]:
# count the number of times a park appears
parks = dict()
for park in park_df['parkname']:
  if park in parks:
    parks[park] += 1
  else:
    parks[park] = 1

i = 0
for park in parks:
  print("Park: {}, appears: {} times".format(park, parks[park]))
  i += 1

print("\nNumber of unique parks is: {}".format(i))

Park: GATEWAY, appears: 6 times
Park: Silver Lake Park, appears: 11 times
Park: Snug Harbor Cultural Center, appears: 3 times
Park: Willowbrook Park, appears: 4 times
Park: LaTourette Park & Golf Course, appears: 4 times
Park: King Fisher Park, appears: 1 times
Park: Great Kills Park, appears: 6 times
Park: Ebs Pond Park, appears: 1 times
Park: Mt. Loretto, appears: 1 times
Park: Highland Park, appears: 13 times
Park: Blue Heron Park, appears: 3 times
Park: Seaton Falls Park, appears: 8 times
Park: Pelham Bay Park, appears: 4 times
Park: Prospect Park, appears: 14 times
Park: Bensonhurst Park, appears: 7 times
Park: Canarsie Park, appears: 4 times
Park: Juniper Valley Playground, appears: 17 times
Park: Miller Field, appears: 2 times
Park: Crescent Beach Park, appears: 2 times
Park: Tottenville Pool, appears: 1 times
Park: nan, appears: 5 times
Park: North Mount Loretto State Forest, appears: 2 times
Park: Forest Park, appears: 48 times
Park: Pugsley Creek Park, appears: 1 times
Park: 

In [7]:
# Creating new column to reflect the zip code the lattitude and longitude correspond to
park_df['Zipcode'] = Zipcode
park_df.sample(10)

,the_geom,gispropnum,parkname,type,Zipcode
767,-73.94507607184202 40.80508330025785,M058,Marcus Garvey Park,1/2-Mile,10026
790,-73.92585257694762 40.66818104800156,B054,Lincoln Terrace / Arthur S. Somers Park,1/2-Mile,11212
490,-74.03051362651266 40.63968915050607,B066,Owl's Head Park,1/2-Mile,11220
403,-73.78281433531666 40.74377961566004,Q300A,Kissena Corridor Park,1/2-Mile,11365
992,-74.24452591261284 40.4990572482358,R006,Conference House Park,1/2-Mile,10307
621,-73.9735626848425 40.79786695363505,M071,Riverside Park,1/2-Mile,10025
316,-74.00456260892472 40.67366148457436,B126,Red Hook Recreation Area,1/2-Mile,11231
957,-74.1353585885055 40.64033739407623,R008,Faber Park,1/2-Mile,10310
159,-73.8994964011082 40.83513639912675,X010,Crotona Park,1/2-Mile,10459
121,-73.94516363533944 40.68922121240117,B088,Herbert Von King Park,1/2-Mile,11216


In [8]:
# checking how many park names to add
null_cols = park_df['parkname'].isnull()
null_parknames = park_df[park_df['parkname'].isnull()][null_cols]
name = null_parknames['the_geom']
print(name)

59     -74.23184858771734 40.514615964676366
65      -74.01786996470521 40.70789628320985
68     -73.94641081474278 40.753677032787174
391       -74.0176875920072 40.7077237221033
808     -73.92982721113754 40.84807555171158
Name: the_geom, dtype: object


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [0]:
# fixing null park names (most likely not necessary)
park_df.loc[park_df.index == 59, 'parkname'] = 'Long Pond Park'
park_df.loc[park_df.index == 65, 'parkname'] = 'Battery Park City Parks'
park_df.loc[park_df.index == 68, 'parkname'] = 'Queensbridge Park'
park_df.loc[park_df.index == 391, 'parkname'] = 'Battery Park City Parks'
park_df.loc[park_df.index == 808, 'parkname'] = 'Highbridge Park'


In [10]:
# get counts of unique zipcodes
unique_zipcodes = dict()
for zipcode in park_df['Zipcode']:
  if zipcode in unique_zipcodes:
    unique_zipcodes[zipcode] += 1
  else:
    unique_zipcodes[zipcode] = 1

for zipcode in unique_zipcodes:
  print("Zipcode: {}, Count: {}".format(zipcode, unique_zipcodes[zipcode]))

Zipcode: 11430, Count: 5
Zipcode: 11414, Count: 1
Zipcode: 10301, Count: 14
Zipcode: 10314, Count: 4
Zipcode: 10306, Count: 17
Zipcode: 10308, Count: 3
Zipcode: 10305, Count: 10
Zipcode: 10307, Count: 9
Zipcode: 11385, Count: 12
Zipcode: 10312, Count: 3
Zipcode: 10466, Count: 8
Zipcode: 10805, Count: 2
Zipcode: 11215, Count: 10
Zipcode: 11214, Count: 14
Zipcode: 11236, Count: 4
Zipcode: 10461, Count: 1
Zipcode: 11379, Count: 17
Zipcode: 11415, Count: 5
Zipcode: 10473, Count: 9
Zipcode: 10280, Count: 6
Zipcode: 11106, Count: 7
Zipcode: 11362, Count: 3
Zipcode: 10026, Count: 23
Zipcode: 10002, Count: 30
Zipcode: 10457, Count: 12
Zipcode: 10458, Count: 5
Zipcode: 11354, Count: 4
Zipcode: 10003, Count: 5
Zipcode: 11356, Count: 7
Zipcode: 10162, Count: 20
Zipcode: 11378, Count: 5
Zipcode: 10468, Count: 13
Zipcode: 10455, Count: 14
Zipcode: 10110, Count: 5
Zipcode: 11357, Count: 1
Zipcode: 11364, Count: 8
Zipcode: 11231, Count: 22
Zipcode: 11216, Count: 4
Zipcode: 11206, Count: 3
Zipcode: 10

In [11]:
# create a new dataframe showing the number of parks at each zipcode
parks_df = pd.DataFrame.from_dict(unique_zipcodes, orient='index', columns=['NumParks'])
parks_df.sample(5)

,NumParks
10475,1
11367,7
11423,1
10456,2
11365,7


In [12]:
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
# Export data
parks_df.to_csv('parks_df.csv', header=True)
!cp parks_df.csv drive/My\ Drive/ #saving to drive